# Life Cycle Assessment - TSAS scenarios

> **⚠ This notebook has been developed with the AeroMAPS version v0.7.1-beta for obtaining the paper results. However, this notebook has been or could be modified in order to be executable with the latest versions of AeroMAPS, which sometimes leads to different results compared to the ones from the paper, due to some models' modifications. In order to retrieve the results of the paper, one can use the v0.7.1-beta version associated with the original notebook.**

The LCA module performs an environmental assessment of the scenarios using data from both AeroMAPS (e.g., fuel combustion emission factors) and the ecoinvent database (for background processes such as electricity generation). In this case study, the environmental profiles of the various fuel production pathways (biofuels and electrofuels) are entirely based on *ecoinvent* data (completed by *premise*) rather than AeroMAPS models. In particular, the some environmental characteristics of the fuel pathways provided in the `energy_inputs.yaml` files are overrode by ecoinvent data in the LCA module (for example the mean CO2 emission factor). While this approach ensures broader coverage of environmental processes, it may also lead to results that are not fully consistent with those generated by AeroMAPS’ core impact models.

## Load modules

First, the user has to load the framework and generate a process.

In [ ]:
# --- Import libraries ---
%matplotlib widget
import pandas as pd
import brightway2 as bw
import lca_algebraic as agb
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import math
import collections
import time
import sys
sys.path.insert(0, './utils/')
from plots import plot_stacked_evolution_subplots
from aeromaps import create_process
plt.style.use("bmh")

## Scenario IS0

In [ ]:
# --- Set AeroMAPS models and create process for scenario IS0 ---
# Note: first call to LCA module takes a dozen minutes depending on CPU, as it will install ecoinvent/premise databases (unless previsouly installed)
# and parametrize the LCA model declared in LCA configuration file (each time the kernel is restarted).
process = create_process(
    configuration_file="data/config_files/config_is0medium_lca.yaml",
)

# For check only: check which LCA databases were installed
list(bw.databases)

In [ ]:
# --- Run assessment ---
start_time = time.time()
process.compute()
process.write_json()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# --- Outputs ---
process_data_vector_outputs_scenario_0 = process.data["vector_outputs"]
process_data_float_inputs_scenario_0 = process.data["float_inputs"]
process_data_climate_scenario_0 = process.data["climate_outputs"]
lca_outputs_scenario_0 = process.data["lca_outputs"]
lca_outputs_scenario_0

In [ ]:
# --- Plot results ---
plt.close()
plot_stacked_evolution_subplots(lca_outputs_scenario_0)

## Scenario IS1

In [ ]:
# --- Create process for scenario IS1 medium ---
process = create_process(
    configuration_file="data/config_files/config_is1medium_lca.yaml",
)

In [ ]:
# --- Run assessment ---
start_time = time.time()
process.compute()
process.write_json()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# --- Outputs ---
process_data_vector_outputs_scenario_1 = process.data["vector_outputs"]
process_data_float_inputs_scenario_1 = process.data["float_inputs"]
process_data_climate_scenario_1 = process.data["climate_outputs"]
lca_outputs_scenario_1 = process.data["lca_outputs"]
lca_outputs_scenario_1

In [ ]:
# --- Plot results ---
plt.close()
plot_stacked_evolution_subplots(lca_outputs_scenario_1)

## Scenario IS2

In [ ]:
# --- Create process for scenario IS2 medium ---
process = create_process(
    configuration_file="data/config_files/config_is2medium_lca.yaml",
)

In [ ]:
# --- Run assessment ---
start_time = time.time()
process.compute()
process.write_json()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# --- Outputs ---
process_data_vector_outputs_scenario_2 = process.data["vector_outputs"]
process_data_float_inputs_scenario_2 = process.data["float_inputs"]
process_data_climate_scenario_2 = process.data["climate_outputs"]
lca_outputs_scenario_2 = process.data["lca_outputs"]
lca_outputs_scenario_2

In [ ]:
# --- Plot results ---
plt.close()
plot_stacked_evolution_subplots(lca_outputs_scenario_2)

## Scenario IS3

In [ ]:
# --- Create process for scenario IS3 medium ---
process = create_process(
    configuration_file="data/config_files/config_is3medium_lca.yaml",
)

In [ ]:
# --- Run assessment ---
start_time = time.time()
process.compute()
process.write_json()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# --- Outputs ---
process_data_vector_outputs_scenario_3 = process.data["vector_outputs"]
process_data_float_inputs_scenario_3 = process.data["float_inputs"]
process_data_climate_scenario_3 = process.data["climate_outputs"]
lca_outputs_scenario_3 = process.data["lca_outputs"]
lca_outputs_scenario_3

In [ ]:
# --- Plot results ---
plt.close()
plot_stacked_evolution_subplots(lca_outputs_scenario_3)

# Postprocessing - From midpoints to endpoints

In [ ]:
# Get all endpoints methods
methods = [
    m for m in agb.findMethods("", mainCat="ReCiPe 2016 v1.03, endpoint (H)") if "total" not in m[1]
]
methods_custom = [m for m in agb.findMethods("", mainCat="Custom methods") if "total" not in m[1]]

methods_ecosystem = [m for m in methods + methods_custom if "ecosystem quality" in m[1]]
methods_human_health = [m for m in methods + methods_custom if "human health" in m[1]]
methods_resources = [m for m in methods + methods_custom if "natural resources" in m[1]]

In [ ]:
# Remove duplicate methods (i.e. ReCiPe methods which are replaced by a custom method, if both are defined in the configuration file)
methods_dict = {
    "ecosystem quality": methods_ecosystem,
    "human health": methods_human_health,
    "natural resources": methods_resources,
}

for name, methods_list in methods_dict.items():
    methods_to_remove = []
    for m in methods_list:
        if m[0] == "Custom methods":
            methods_to_remove.append(("ReCiPe 2016 v1.03, endpoint (H)", name, m[2]))
    methods_dict[name] = [m for m in methods_list if m not in methods_to_remove]

In [ ]:
# Add original LCIA methods for climate change to get difference involved by non-CO2
# This is more convenient than splitting the results by 'phase' but will require post-processing by hand
methods_dict["ecosystem quality"].append(
    (
        "ReCiPe 2016 v1.03, endpoint (H)",
        "ecosystem quality",
        "climate change: freshwater ecosystems",
    )
)
methods_dict["ecosystem quality"].append(
    (
        "ReCiPe 2016 v1.03, endpoint (H)",
        "ecosystem quality",
        "climate change: terrestrial ecosystems",
    )
)
methods_dict["human health"].append(
    ("ReCiPe 2016 v1.03, endpoint (H)", "human health", "climate change: human health")
)

In [ ]:
# Create function to get the data for each scenario
def get_scenario_data(scenario, year):
    scenario_data_vector = globals()[f"process_data_vector_outputs_scenario_{scenario}"]
    scenario_data_float = globals()[f"process_data_float_inputs_scenario_{scenario}"]
    scenario_data_climate = globals()[f"process_data_climate_scenario_{scenario}"]

    params_dict = dict(
        model="remind",
        pathway="SSP2_NPi",
        rpk_long_range=scenario_data_vector["rpk_long_range"][year],
        rpk_medium_range=scenario_data_vector["rpk_medium_range"][year],
        rpk_short_range=scenario_data_vector["rpk_short_range"][year],
        fossil_kerosene_mass_consumption=scenario_data_vector["fossil_kerosene_mass_consumption"][
            year
        ],
        generic_biofuel_mass_consumption=scenario_data_vector["generic_biofuel_mass_consumption"][
            year
        ]
        if "generic_biofuel_mass_consumption" in scenario_data_vector
        else 0.0,
        electrofuel_mass_consumption=scenario_data_vector["electrofuel_mass_consumption"][year]
        if "electrofuel_mass_consumption" in scenario_data_vector
        else 0.0,
        hydrogen_electrolysis_mass_consumption=scenario_data_vector[
            "hydrogen_electrolysis_mass_consumption"
        ][year]
        if "hydrogen_electrolysis_mass_consumption" in scenario_data_vector
        else 0.0,
        fossil_kerosene_lhv=scenario_data_float["fossil_kerosene_lhv"],
        generic_biofuel_lhv=scenario_data_float["generic_biofuel_lhv"],
        electrofuel_lhv=scenario_data_float["electrofuel_lhv"],
        fossil_kerosene_emission_index_nox=scenario_data_float[
            "fossil_kerosene_emission_index_nox"
        ],
        fossil_kerosene_emission_index_sulfur=scenario_data_float[
            "fossil_kerosene_emission_index_sulfur"
        ],
        fossil_kerosene_emission_index_soot=scenario_data_float[
            "fossil_kerosene_emission_index_soot"
        ],
        generic_biofuel_emission_index_nox=scenario_data_float[
            "generic_biofuel_emission_index_nox"
        ],
        generic_biofuel_emission_index_sulfur=scenario_data_float[
            "generic_biofuel_emission_index_sulfur"
        ],
        generic_biofuel_emission_index_soot=scenario_data_float[
            "generic_biofuel_emission_index_soot"
        ],
        electrofuel_emission_index_nox=scenario_data_float["electrofuel_emission_index_nox"],
        electrofuel_emission_index_sulfur=scenario_data_float["electrofuel_emission_index_sulfur"],
        electrofuel_emission_index_soot=scenario_data_float["electrofuel_emission_index_soot"],
        hydrogen_electrolysis_emission_index_nox=scenario_data_float[
            "hydrogen_electrolysis_emission_index_nox"
        ],
        total_aircraft_distance=scenario_data_climate["total_aircraft_distance"][year],
        fuel_effect_correction_contrails=scenario_data_vector["fuel_effect_correction_contrails"][
            year
        ],
        elec_solar_share=1.0,
        year=year,
    )

    # Check if all parameters are provided
    missing_keys = set(agb.params.all_params().keys()) - set(params_dict.keys())
    extra_keys = set(agb.params.all_params().keys()) - set(params_dict.keys())

    # Raise errors for missing or extra keys
    if missing_keys:
        raise KeyError(f"Parameters are missing: {missing_keys}")
    if extra_keys:
        raise KeyError(f"Two many parameters: {extra_keys}")

    return params_dict

In [ ]:
# Settings
year = 2050  # In what year
scenario_numbers = [0, 1, 2, 3]  # Which scenarios (is0 to is3)

# Initialize dictionaries to hold dataframes for each method
dfs = {}

# This part is not computationnaly efficient and shoud be improved in the future...
for method_name, method in methods_dict.items():
    df = pd.DataFrame()
    for scenario in scenario_numbers:
        params_dict = get_scenario_data(scenario, year)

        res = agb.compute_impacts(
            process.models["life_cycle_assessment"].model,
            method,
            **params_dict,
        )

        # Rename the index for the current result
        res = res.rename(index={"model": f"scenario {scenario}"})

        # Concatenate the result to the DataFrame
        df = pd.concat([df, res], axis=0, ignore_index=False)

    # Normalize by the values of scenario 0
    # scenario_0_values = df.loc[df.index == 'scenario 0']
    # df = df.divide(scenario_0_values.values.sum())

    # Store the dataframe in the dictionary
    dfs[method_name] = df

In [ ]:
# Save to excel file
for method_name in methods_dict.keys():
    dfs[method_name].to_excel(f"tsas_endpoints_contributions_{method_name}.xlsx")

In [ ]:
# Modify the xlsx file at your wish for better plots, e.g. by merging low impact categories together and renaming the categories.

In [ ]:
# Reimport the data
dfs = {
    method_name: pd.read_excel(f"tsas_endpoints_contributions_{method_name}.xlsx", index_col=0)
    for method_name in methods_dict.keys()
}
combined_df = pd.concat(dfs, names=["Method", "Scenario"])  # .reset_index()#(level=0)
combined_df

In [ ]:
import matplotlib.gridspec as gridspec

# Set up three subplots (one for each endpoint)
clusters = combined_df.index.levels[0]
inter_graph = 0
maxi = np.max(np.sum(combined_df, axis=1))
total_width = len(combined_df) + inter_graph * (len(clusters) - 1)
fig = plt.figure(figsize=(total_width, 6))

# Plot properties
gridspec.GridSpec(1, total_width)
axes = []
palette = sns.color_palette("tab10")
# palette.insert(1, palette[1])  # Duplicate color for second position (climate change CO2) for third position (climate change Non-CO2)
# hatches = [''] * len(combined_df.index)
# hatches[2] = '//'

ax_position = 0
for cluster in clusters:
    subset = combined_df.loc[cluster]
    ax = subset.plot(
        kind="bar",
        stacked=True,
        width=0.8,
        ax=plt.subplot2grid((1, total_width), (0, ax_position), colspan=len(subset.index)),
        color=palette,
        alpha=0.8,
        edgecolor="0.2",
    )
    axes.append(ax)
    ax.set_title(cluster.title(), fontsize=15)
    ax.set_xlabel("")
    # ax.set_ylim(0,maxi*1.1)
    ax_position += len(subset.index) + inter_graph
    ax.tick_params(axis="x", rotation=0, labelsize=11)

    tick_labels = [label.get_text() for label in ax.get_xticklabels()]
    wrapped_labels = [
        "\n(".join([label.split("(")[0], label.split("(")[1]]) if "(" in label else label
        for label in tick_labels
    ]
    ax.set_xticklabels(wrapped_labels)
    ax.set_axisbelow(True)

    # Apply hatches to the specific segment of the stacks
    # for bar_group, hatch_pattern in zip(ax.containers, hatches[:len(subset.columns)]):
    #    for bar in bar_group:
    #        bar.set_hatch(hatch_pattern)

for i in range(0, len(clusters)):
    axes[i].legend().set_visible(False)
for i in range(1, len(clusters) - 1):
    axes[i].set_yticklabels("")
axes[-1].yaxis.tick_right()
axes[-1].tick_params(axis="y", labelsize=13)
axes[0].tick_params(axis="y", labelsize=13)
axes[0].set_ylabel("Impacts relative to IS0", fontsize=15)

# Collect legend labels from all plots.
entries = collections.OrderedDict()
for ax in axes:
    for handle, label in zip(*axes[0].get_legend_handles_labels()):
        label_name = label.replace("_", " ").title()
        entries[label_name] = handle
legend = fig.legend(
    entries.values(),
    entries.keys(),
    loc="lower center",
    bbox_to_anchor=(0.5, 0),
    ncol=2,
    fontsize=13,
    title="Midpoint Category",
    title_fontsize=14,
)

# Set tight layout while keeping legend in the screen
bbox = legend.get_window_extent(fig.canvas.get_renderer()).transformed(fig.transFigure.inverted())
fig.tight_layout(rect=(0, bbox.y1, 1, 1), h_pad=0.5, w_pad=0.5)

plt.show()